# 행렬 곱

In [1]:
arr1 = [
    [1, 2],
    [3, 4],
]

In [2]:
arr2 = [
    [5, 6],
    [7, 8],
]

In [4]:
list(zip(*arr2))

# zip([5, 6], [7, 8]) # generator

[(5, 7), (6, 8)]

In [6]:
mylist1 = ['a', 'b', 'c']
mylist2 = ['x', 'y', 'z']

for i in range(len(mylist1)):
    v1, v2 = mylist1[i], mylist2[i]
    print(v1, v2)

for (v1, v2) in zip(mylist1, mylist2):
    print(v1, v2)

print('----')
mylist = [mylist1, mylist2]
print(mylist)

for (v1, v2) in zip(*mylist):
    print(v1, v2)

a x
b y
c z
a x
b y
c z
----
[['a', 'b', 'c'], ['x', 'y', 'z']]
a x
b y
c z


In [8]:
result_list = []
for row1 in arr1:
    row_list = []
    for row2 in zip(*arr2):
        result = sum(i*j for i, j in zip(row1, row2))
        row_list.append(result)
    result_list.append(row_list)
result_list

[[19, 22], [43, 50]]

In [14]:
[[sum(i*j for i, j in zip(row1, row2))
      for row2 in zip(*arr2)]
     for row1 in arr1]

[[19, 22], [43, 50]]